In [42]:
import pandas as pd
import hvplot.pandas
import numpy as np

In [43]:
df = pd.read_csv('callsforservice.csv')
df.head()

,INCIDENT_NUM,DISP_CODE,DISPOSITION,FINAL_RADIO_CODE,FINAL_CALL_TYPE,CALL_RECEIVED,HUNDREDBLOCKADDR,GRID
0,201502108490,N,NO ACTION REQUIRED,647V,SUSPICIOUS PERSON IN VEHICLE,11/2/2015 9:16:27AM,154XX N 29TH ST,DF34
1,201502108749,N,NO ACTION REQUIRED,418T,TRESPASSING,11/2/2015 9:58:57AM,96XX N 10TH AVE,CH26
2,201502108648,O,OTHER (SUPP/FALSE ALARM/FI/LOUD PARTY),459A,BURGLARY ALARM,11/2/2015 9:42:42AM,38XX E WATKINS ST,AH36B
3,201502108757,N,NO ACTION REQUIRED,418T,TRESPASSING,11/2/2015 10:02:11AM,36XX W CAMELBACK RD,BI20
4,201502108592,N,NO ACTION REQUIRED,900,CHECK WELFARE,11/2/2015 9:30:53AM,40XX N 44TH AVE,BG18


In [44]:
with open('apikey.txt', 'r') as door:
    key = door.read()

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2171125 entries, 0 to 2171124
Data columns (total 8 columns):
INCIDENT_NUM        int64
DISP_CODE           object
DISPOSITION         object
FINAL_RADIO_CODE    object
FINAL_CALL_TYPE     object
CALL_RECEIVED       object
HUNDREDBLOCKADDR    object
GRID                object
dtypes: int64(1), object(7)
memory usage: 132.5+ MB


In [46]:
# tst = df.iloc[:500, :]
# tst.head()

In [47]:
df['HUNDREDBLOCKADDR'] = df.HUNDREDBLOCKADDR.str.replace('XX', '00')
df.dropna(inplace=True)
df = df[df.FINAL_RADIO_CODE.isin(['961', '962', '963', '961H', '962H', '963H'])]
df.loc[:, 'CALL_RECEIVED'] = pd.to_datetime(df.CALL_RECEIVED)
df.set_index('CALL_RECEIVED', inplace=True)

In [48]:
df['month'] = df.index.month
df['day'] = df.index.dayofweek
df['hour'] = df.index.hour

In [57]:
df.head()

,INCIDENT_NUM,DISP_CODE,DISPOSITION,FINAL_RADIO_CODE,FINAL_CALL_TYPE,HUNDREDBLOCKADDR,GRID,month,day,hour
CALL_RECEIVED,,,,,,,,,,
2015-11-01 01:01:07,201502100315,N,NO ACTION REQUIRED,961H,HIT & RUN ACCIDENT NO INJURY,S 67TH AVE & W LOWER BUCKEYE RD,AG13,11,6,1
2015-11-02 08:04:44,201502108062,A,ACCIDENT REPORT,962,ACCIDENT WITH INJURIES,N 49TH DR & W ESCUDA DR,EA17,11,0,8
2015-11-01 10:09:50,201502102088,N,NO ACTION REQUIRED,961,ACCIDENT NO INJURIES,E PARADISE LN & N 32ND ST,DF35,11,6,10
2015-11-01 10:35:06,201502102198,N,NO ACTION REQUIRED,961,ACCIDENT NO INJURIES,4000 W CAMELBACK RD,BI19A,11,6,10
2015-11-01 18:03:38,201502104894,N,NO ACTION REQUIRED,961,ACCIDENT NO INJURIES,3700 W SIERRA ST,DA20,11,6,18


In [58]:
df['ishitrun'] = df.FINAL_RADIO_CODE.isin(['961H', '962H', '963H']).astype(np.int)

In [61]:
hracs = df[df.ishitrun == 1]
lacs = df[df.ishitrun == 0]

In [74]:
lacs.month.hvplot.hist(color='navy').options(alpha=0.7) * hracs.month.hvplot.hist(color='red').options(alpha=0.7)

:Overlay
   .Histogram.I  :Histogram   [month]   (month_count)
   .Histogram.II :Histogram   [month]   (month_count)

In [73]:
lacs.day.hvplot.hist(color='navy').options(alpha=0.7) * hracs.day.hvplot.hist(color='red').options(alpha=0.7)

:Overlay
   .Histogram.I  :Histogram   [day]   (day_count)
   .Histogram.II :Histogram   [day]   (day_count)

In [75]:
lacs.hour.hvplot.hist(color='navy').options(alpha=0.7) * hracs.hour.hvplot.hist(color='red').options(alpha=0.7)

:Overlay
   .Histogram.I  :Histogram   [hour]   (hour_count)
   .Histogram.II :Histogram   [hour]   (hour_count)

In [76]:
lacs.DISPOSITION.value_counts() / len(lacs) * 100

ACCIDENT REPORT                           61.465645
NO ACTION REQUIRED                        31.046012
DEPARTMENTAL REPORT                        7.037190
OTHER (SUPP/FALSE ALARM/FI/LOUD PARTY)     0.230612
TRAFFIC CITATION                           0.220542
Name: DISPOSITION, dtype: float64

In [77]:
hracs.DISPOSITION.value_counts() / len(hracs) * 100

NO ACTION REQUIRED                        51.448047
DEPARTMENTAL REPORT                       24.762867
ACCIDENT REPORT                           23.230065
OTHER (SUPP/FALSE ALARM/FI/LOUD PARTY)     0.483283
TRAFFIC CITATION                           0.075738
Name: DISPOSITION, dtype: float64

In [79]:
lacs.FINAL_CALL_TYPE.value_counts() / len(lacs) * 100

ACCIDENT NO INJURIES      81.196564
ACCIDENT WITH INJURIES    18.588937
ACCIDENT FATALITY          0.214499
Name: FINAL_CALL_TYPE, dtype: float64

In [80]:
hracs.FINAL_CALL_TYPE.value_counts() / len(hracs) * 100

HIT & RUN ACCIDENT NO INJURY        93.573052
HIT & RUN ACCIDENT WITH INJURIES     6.336784
HIT & RUN  FATALITY                  0.090165
Name: FINAL_CALL_TYPE, dtype: float64

In [82]:
len(hracs) / (len(hracs) + len(lacs))

0.21827471108731933

In [84]:
df.GRID.value_counts()

BG27     700
BA27     621
BG13     598
BC19A    586
BI23B    561
DG29     550
CI22     523
BC17A    521
BE11     517
CA21     516
BG17     512
BG19     507
BE31     506
AB16     500
BI29     481
BD11     479
BE38     477
BI31B    477
BG29     472
BG11     466
BC13A    454
BI32     453
BE17     451
BE35     446
CA25     431
BC29A    421
BG21A    413
CA23B    413
BI21     410
CG21     400
        ... 
JD28       1
EA36B      1
FD24       1
FI37       1
EB35B      1
HB16       1
FD34       1
CH32       1
FA35       1
EI34       1
FF40       1
CH18       1
IF20       1
AE14       1
AE03       1
IF26       1
AF20B      1
EB44       1
EJ17       1
EJ36       1
DB27       1
JB24       1
FF42       1
EE40       1
GD18       1
EE28       1
IC12       1
IF29       1
JA28       1
IC14       1
Name: GRID, Length: 1667, dtype: int64